In [ ]:
import tensorflow as tf
import segmentation_models as sm
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import keras
from keras.metrics import MeanIoU

In [ ]:
SIZE_X = 256
SIZE_Y = 256
n_classes = 2

# Loading images

In [ ]:
# Capture training image info as a list
train_images = []

for directory_path in glob.glob("new_images"):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, 1)
        img = cv2.resize(img, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST)
        train_images.append(img)
    
# Convert list to array for machine learning processing        
train_images = np.array(train_images)

# Loading masks

In [ ]:
# Capture mask/label info as a list
train_masks = []

for directory_path in glob.glob("new_masks"):
    for mask_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        mask = cv2.imread(mask_path, 0)
        mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST)
        mask[mask <= 15] = 0
        mask[mask >= 200] = 255
        train_masks.append(mask)
        
# Convert list to array for machine learning processing          
train_masks = np.array(train_masks)

# Preprocessing

In [ ]:
# Encode labels
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
n, h, w = train_masks.shape
train_masks_reshaped = train_masks.reshape(-1, 1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

np.unique(train_masks_encoded_original_shape)

In [ ]:
train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis = 3)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_images, train_masks_input, test_size = 0.10, random_state = 2)

In [ ]:
a, b = np.unique(train_masks[0], return_counts = True)

print("Class values in the dataset are ... ", b)

In [ ]:
from keras.utils import to_categorical
train_masks_cat = to_categorical(train_masks_input, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((train_masks_input.shape[0], train_masks_input.shape[1], train_masks_input.shape[2], n_classes))

test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))

In [ ]:
def dice_coef(y_true, y_pred, smooth=1.0):
    # Flatten the input tensors
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    
    # Calculate intersection and union of the flattened tensors
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    union = tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f)
    
    # Calculate Dice coefficient
    dice_coef = (2.0 * intersection + smooth) / (union + smooth)
    
    return dice_coef

In [ ]:
n_classes = 2
activation = 'softmax'

LR = 0.001
optim =  keras.optimizers.Adam(LR)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), dice_coef]

In [ ]:
def Exp_loss(y_true, y_pred):
  y_pred = tf.nn.softmax(y_pred)

  true = 0.8 * tf.math.exp(-1 * tf.math.multiply(y_true, y_pred))
  wrong = tf.math.exp(-1 * tf.math.multiply((1 - y_true), (1 - y_pred)))
  loss = tf.reduce_mean(true + wrong)
  
  return loss

# Model

In [ ]:
BACKBONE1 = 'resnet101'
preprocess_input1 = sm.get_preprocessing(BACKBONE1)

X_train1 = preprocess_input1(train_images)
X_test1 = preprocess_input1(test_images)

In [ ]:
from keras_unet_collection import models
import tensorflow as tf

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True # enable dynamic GPU memory allocation
session = tf.compat.v1.Session(config=config)

model = models.unet_2d((256, 256, 3), [64, 128, 256, 512, 1024], n_labels = 2,
                      stack_num_down = 2, stack_num_up = 2, backbone='ResNet101',
                      activation='ReLU', output_activation = 'Softmax',
                      batch_norm = True, pool = 'max', unpool = 'bilinear', name = 'unet-main')

model.summary()

In [ ]:
model.compile(optim, Exp_loss, metrics = metrics)

# Training

In [ ]:
history1 = model.fit(X_train1,
          y_train_cat,
          batch_size = 8,
          epochs = 50,
          validation_split = 0.125)

# Evaluation

In [ ]:
y_pred = model.predict(test_images)
y_pred_argmax = np.argmax(y_pred, axis = 3)

In [ ]:
n_classes = 2
IOU_keras = MeanIoU(num_classes = n_classes)
IOU_keras.update_state(test_masks_input[:, :, :, 0], y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

In [ ]:
# To calculate I0U for each class...
values = np.array(IOU_keras.get_weights()).reshape(n_classes, n_classes)
print(values)
class1_IoU = values[0,0] / (values[0,0] + values[0,1] + values[1,0])
class2_IoU = values[1,1] / (values[1,1] + values[1,0]  + values[0,1])

print("IoU for class1 is: ", class1_IoU)
print("IoU for class2 is: ", class2_IoU)

In [ ]:
loss = history1.history['loss']
val_loss = history1.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history1.history['iou_score']
val_acc = history1.history['val_iou_score']

plt.plot(epochs, acc, 'y', label='Training IOU')
plt.plot(epochs, val_acc, 'r', label='Validation IOU')
plt.title('Training and validation IOU')
plt.xlabel('Epochs')
plt.ylabel('IOU')
plt.legend()
plt.show()

In [ ]:
# Test some random images
import random
test_img_number = random.randint(0, len(X_test1) - 1)
print(test_img_number)
test_img = X_test1[test_img_number]
ground_truth = test_masks_input[test_img_number]
test_img_input = np.expand_dims(test_img, 0)

test_img_input1 = preprocess_input1(test_img_input)

test_pred1 = model.predict(test_img_input1)
test_prediction1 = np.argmax(test_pred1, axis=3)[0,:,:]

plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[:,:,0], cmap='gray')
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth[:,:,0], cmap='jet')
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(test_prediction1, cmap='jet')
plt.show()